In [105]:
%pip install hbase_rest_py

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [100]:
import hbase

In [101]:
from hbase.rest_client import HBaseRESTClient
from hbase.admin import HBaseAdmin
client = HBaseRESTClient(['http://node1:8000'])
admin = HBaseAdmin(client)

In [102]:
def fetch_by_date_hbase(table, date_to_process):
    from hbase.scan_filter_helper import build_prefix_filter
    from hbase.scan import Scan

    date_filter = build_prefix_filter(date_to_process)

    scan = Scan(client)
    result = scan.scan(table, date_filter)
    
    data = [
        {
            (cell["column"]).decode().split(":")[1] : (cell["$"]).decode()
            for cell in row["cell"]
        }
        for row in result[1]["row"]
    ]
    
    scan.delete_scanner()
    
    return data

In [103]:
fetch_by_date_hbase("veh_stats", "20221231")[:3]

[{'Brigade': '1',
  'Date': '20221231',
  'Lines': ' ',
  'VehicleNumber': '9543',
  'AnalysisDate': '2023-01-08T13:59:57.220175',
  'BusDataSource': '/user/kpk/speeds/20221231.avro',
  'AvgSpeed_mps': '3.3807907042450798',
  'TotalDist_km': '1.0808255407932414',
  'TotalTime_h': '0.09166666666666666'},
 {'Brigade': '51',
  'Date': '20221231',
  'Lines': '102',
  'VehicleNumber': '7220',
  'AnalysisDate': '2023-01-08T13:59:57.216055',
  'BusDataSource': '/user/kpk/speeds/20221231.avro',
  'AvgSpeed_mps': '1.4352837722561818',
  'TotalDist_km': '0.37440960643673804',
  'TotalTime_h': '5.036944444444444'},
 {'Brigade': '51',
  'Date': '20221231',
  'Lines': '102',
  'VehicleNumber': '7222',
  'AnalysisDate': '2023-01-08T13:59:57.218803',
  'BusDataSource': '/user/kpk/speeds/20221231.avro',
  'AvgSpeed_mps': '0.024289016239940675',
  'TotalDist_km': '0.003949580307537217',
  'TotalTime_h': '0.052222222222222225'}]

In [106]:
for table in ["veh_stats", "line_stats", "bus_weather_stats", "bus_weather_line_stats"]:
    with open(f"hbase_table_{table}.json", "w") as file:
        file.write(fetch_by_date_hbase(table, "20221231"))

TypeError: write() argument must be str, not list